In [1]:
import numpy as np
from tqdm import tqdm
import torch
from torch_geometric.data import Data

In [2]:
from tqdm import trange


with open('txt_graphs/indochina-2004.graph-txt', 'r') as f:
    line = f.readline()
    line_tot = int(line.split()[0])
    print("{} lines".format(line_tot))
    out_nodes = [0] * line_tot
    in_nodes = [0] * line_tot
    out_degree = [0] * line_tot
    for i in trange(line_tot):
        line = f.readline()
        if line[0] == '\n':
            in_ = np.array([i])
            out_ = np.array([i])
        else:
            in_ = np.fromstring(line, dtype=int, sep=' ')
            out_ = np.ones(len(in_), dtype=int) * i
        out_nodes[i] = out_
        in_nodes[i] = in_
        out_degree[i] = len(out_)


out_nodes = np.hstack(out_nodes)
in_nodes = np.hstack(in_nodes)

  0%|          | 6548/7414866 [00:00<01:53, 65479.91it/s]

7414866 lines


100%|██████████| 7414866/7414866 [01:26<00:00, 85300.80it/s] 


In [4]:
max(out_degree)

6985

In [5]:
unique_elements, in_degree = np.unique(in_nodes, return_counts=True)

In [6]:
sorted(in_degree)[-10:]

[101479,
 101500,
 101948,
 102027,
 150208,
 154369,
 176790,
 178007,
 179280,
 256426]

In [7]:
in_degree[:10]

array([1, 2, 5, 3, 6, 1, 1, 4, 2, 1])

In [10]:
np.array(out_degree).shape

(7414866,)

In [11]:
np.array(in_degree).shape

(7414677,)

In [12]:
in_nodes.shape

(195418438,)

In [13]:
out_nodes.shape

(195418438,)

In [15]:
unique_elements[-5:]

array([7414861, 7414862, 7414863, 7414864, 7414865])

In [17]:
temp = np.setdiff1d(np.arange(7414866), unique_elements)

In [18]:
temp.shape

(189,)

In [19]:
temp[:5]

array([ 51252, 308657, 643079, 784345, 894429])

In [23]:
a = np.zeros(len(out_degree))
a[unique_elements] = in_degree

In [24]:
a.shape

(7414866,)

In [25]:
edge_index = torch.tensor(np.vstack((out_nodes, in_nodes)), dtype=torch.long)
x = torch.tensor(np.vstack((out_degree, a)).T, dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

In [26]:
data.num_nodes

7414866

In [27]:
data.num_edges

195418438

In [ ]:
data.contains_isolated_nodes()

In [ ]:
data.contains_self_loops()

In [ ]:
data.is_directed()

In [28]:
def train(loader, data, model, optimizer, device='cuda'):
    model.train()
    total_loss = 0
    for subset in loader:
        optimizer.zero_grad()
        loss = model.loss(data.edge_index, subset.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [15]:
from torch.utils.data import DataLoader
from torch_geometric.nn import Node2Vec

loader = DataLoader(torch.arange(data.num_nodes), batch_size=128, shuffle=True)
device = 'cuda'
model = Node2Vec(data.num_nodes, embedding_dim=64, walk_length=20,
                 context_size=10, walks_per_node=10)
model, data = model.to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in trange(1, 20):
    loss = train(loader, data, model, optimizer)
    print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))

  5%|▌         | 1/19 [00:47<14:08, 47.15s/it]

Epoch: 01, Loss: 1.6546


 11%|█         | 2/19 [01:28<12:53, 45.48s/it]

Epoch: 02, Loss: 0.8097


 16%|█▌        | 3/19 [02:10<11:48, 44.28s/it]

...

Epoch: 17, Loss: 0.6668


 95%|█████████▍| 18/19 [12:32<00:41, 41.50s/it]

Epoch: 18, Loss: 0.6650


100%|██████████| 19/19 [13:14<00:00, 41.50s/it]

Epoch: 19, Loss: 0.6620


In [16]:
sub = torch.tensor([0, 1, 2])
sub = sub.to(device)
model.forward(sub)

tensor([[ 2.0825e-01,  1.3607e-01,  4.9600e-02, -6.0241e-01, -2.4672e-03,
          8.2338e-02,  1.7288e-01,  5.9137e-02, -4.6599e-02,  4.9159e-01,
         -5.9120e-01, -3.9939e-01,  4.5430e-02,  3.9232e-01,  3.9860e-01,
         -8.2821e-02,  4.7175e-01,  2.8038e-01, -6.8810e-02, -2.1342e-01,
          3.5228e-02,  1.9292e-01, -1.5499e-01, -3.8864e-01, -3.8246e-01,
          6.5634e-02,  6.4747e-01,  2.6628e-01,  3.4164e-01, -2.5770e-01,
          2.2271e-01, -3.8287e-01,  3.1442e-01, -3.3332e-01,  2.1094e-01,
         -2.5800e-01,  1.1145e-03,  3.3403e-01, -2.1289e-01, -2.0785e-01,
          4.7928e-01,  2.2601e-01,  1.7835e-01, -7.7802e-01,  1.4940e-01,
         -2.7403e-01, -1.6797e-01,  5.2571e-01,  3.8486e-01,  2.6403e-01,
          4.7272e-02, -6.3045e-01, -6.5102e-02,  4.0672e-01,  1.7066e-01,
          3.5464e-01,  1.7278e-01,  8.0207e-02,  4.3153e-01, -1.8736e-01,
         -4.3730e-01, -1.4869e-01, -3.3755e-01, -1.3141e-01],
        [-2.5264e-01, -2.4382e-01,  1.6596e-01, -2

In [17]:
data.x.shape

torch.Size([325557, 2])

In [18]:
import faiss

n = data.x.shape[0]
sub = torch.tensor([i for i in range(n)])
sub = sub.to(device)
xb = model.forward(sub).cpu().detach().numpy()
nb, d = xb.shape
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
325557


In [19]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[     0     81    189    269]
 [     1     57    274    255]
 [     2      7    187    204]
 [     3     24    187    114]
 [     4 295695 297677 290495]]
[[0.        3.123289  3.244448  3.2812066]
 [0.        2.9738853 2.9919443 3.007856 ]
 [0.        3.922614  4.097231  4.4069276]
 [0.        3.733567  3.7458014 3.8158584]
 [0.        3.7408037 3.8452568 3.8463328]]


In [20]:
help(Node2Vec)

Help on class Node2Vec in module torch_geometric.nn.models.node2vec:

class Node2Vec(torch.nn.modules.module.Module)
 |  The Node2Vec model from the
 |  `"node2vec: Scalable Feature Learning for Networks"
 |  <https://arxiv.org/abs/1607.00653>`_ paper where random walks of
 |  length :obj:`walk_length` are sampled in a given graph, and node embeddings
 |  are learned via negative sampling optimization.
 |  
 |  Args:
 |      num_nodes (int): The number of nodes.
 |      embedding_dim (int): The size of each embedding vector.
 |      walk_length (int): The walk length.
 |      context_size (int): The actual context size which is considered for
 |          positive samples. This parameter increases the effective sampling
 |          rate by reusing samples across different source nodes.
 |      walks_per_node (int, optional): The number of walks to sample for each
 |          node. (default: :obj:`1`)
 |      p (float, optional): Likelihood of immediately revisiting a node in the
 |     

In [23]:
model2 = Node2Vec(data.num_nodes, embedding_dim=32, walk_length=30,
                 context_size=20, walks_per_node=10, p=1, q=2)
model2 = model2.to(device)

for epoch in trange(1, 30):
    loss = train(loader, data, model2)
    print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))



  0%|          | 0/29 [00:00<?, ?it/s]

  3%|▎         | 1/29 [00:46<21:43, 46.55s/it]

Epoch: 01, Loss: 3.6646




  7%|▋         | 2/29 [01:33<20:57, 46.56s/it]

Epoch: 02, Loss: 3.6648




 10%|█         | 3/29 [02:19<20:10, 46.57s/it]

...

Epoch: 27, Loss: 3.6647




 97%|█████████▋| 28/29 [21:43<00:46, 46.52s/it]

Epoch: 28, Loss: 3.6648




100%|██████████| 29/29 [22:30<00:00, 46.53s/it]

Epoch: 29, Loss: 3.6648


In [31]:
model3 = Node2Vec(data.num_nodes, embedding_dim=32, walk_length=30,
                 context_size=20, walks_per_node=10, p=1, q=2,
                 num_negative_samples=40)
optimizer = torch.optim.Adam(model3.parameters(), lr=0.005)

model3 = model3.to(device)

for epoch in trange(1, 30):
    loss = train(loader, data, model3, optimizer)
    print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))







  0%|          | 0/29 [00:00<?, ?it/s]





  3%|▎         | 1/29 [00:45<21:09, 45.34s/it]

Epoch: 01, Loss: 1.3618








  7%|▋         | 2/29 [01:30<20:24, 45.35s/it]

Epoch: 02, Loss: 0.8059








 10%|█         | 3/29 [02:16<19:39, 45.35s/it]

...

Epoch: 27, Loss: 0.6290








 97%|█████████▋| 28/29 [21:10<00:45, 45.43s/it]

Epoch: 28, Loss: 0.6288








100%|██████████| 29/29 [21:55<00:00, 45.43s/it]

Epoch: 29, Loss: 0.6286


In [54]:
def train_search(data, model, device='cuda'):
    n = data.x.shape[0]
    sub = torch.tensor([i for i in range(n)])
    sub = sub.to(device)
    # get embedding for the training nodes
    xb = model.forward(sub).cpu().detach().numpy()
    # train similarity search model
    nb, d = xb.shape
    index = faiss.IndexFlatL2(d)   # build the index
    print("Index trained: {}".format(index.is_trained))
    index.add(xb)                  # add vectors to the index
    print("Index total: {}".format(index.ntotal))
    return xb, index

In [56]:
xb, ind = train_search(data, model3)
print(xb.shape)
k = 4
D, I = ind.search(xb[:5], k) # sanity check

Index trained: True
Index total: 325557
(325557, 32)


In [51]:
I

array([[     0,     81,    189,    269],
       [     1,     57,    274,    255],
       [     2,      7,    187,    204],
       [     3,     24,    187,    114],
       [     4, 295695, 297677, 290495]])

In [57]:
model4 = Node2Vec(data.num_nodes, embedding_dim=64, walk_length=30,
                 context_size=20, walks_per_node=10, p=1, q=2,
                 num_negative_samples=50)
optimizer = torch.optim.Adam(model4.parameters(), lr=0.001)
model4 = model4.to(device)

for epoch in trange(1, 50):
    loss = train(loader, data, model4, optimizer)
    print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))







  0%|          | 0/49 [00:00<?, ?it/s]





  2%|▏         | 1/49 [00:56<45:23, 56.74s/it]

Epoch: 01, Loss: 3.4679








  4%|▍         | 2/49 [01:53<44:27, 56.76s/it]

Epoch: 02, Loss: 2.1328








  6%|▌         | 3/49 [02:50<43:31, 56.78s/it]

...

Epoch: 47, Loss: 0.5854








 98%|█████████▊| 48/49 [45:26<00:56, 56.81s/it]

Epoch: 48, Loss: 0.5846








100%|██████████| 49/49 [46:22<00:00, 56.82s/it]

Epoch: 49, Loss: 0.5841


In [58]:
xb, ind = train_search(data, model3)
k = 4
D, I = ind.search(xb[:5], k) # sanity check

Index trained: True
Index total: 325557


In [59]:
I

array([[  0,   4,  75, 272],
       [  1, 121, 180,  56],
       [  2, 243, 175,  50],
       [  3, 298,  36, 272],
       [  4,   9, 208,  65]])

In [60]:
D, I = ind.search(xb[-5:], k)

In [61]:
I

array([[325552, 325551, 325550,  20317],
       [325553, 182061, 315451, 325554],
       [325554, 315451, 325553, 182061],
       [325555, 316067, 289204, 289442],
       [325556, 289145, 289345, 289213]])

In [69]:
D, I = ind.search(xb[-1].reshape(1, -1), k)

In [73]:
I.shape

(1, 4)

In [74]:
np.save('embeddings.npy', xb)

In [76]:
temp = np.load('embeddings.npy')

In [78]:
temp.shape

(325557, 32)

In [62]:
import linecache
# >>> linecache.getline('/etc/passwd', 4)

In [79]:
import linecache


def check(nodes, k, emb, ind, file):
    """ 
    emb is a 2-d numpy array of embeddings
    """
    if len(nodes) == 1:
        D, I = ind.search(emb[nodes].reshape(1, -1), k)
    else:
        D, I = ind.search(emb[nodes], k)
    for row in I:
        print("{} nearest neighbour of node {}".format(k, row[0]))
        for node in row[1:]:
            print("  " + linecache.getline(file, node + 1))